In [1]:
import pandas as pd
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
from pprint import pprint


In [2]:
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')

In [3]:
df = pd.read_csv("data/open_tasks_Corn_EKV_DE.csv")
data = df["description"]

In [46]:
import spacy

# Load the German language model
nlp = spacy.load("de_core_news_sm")

def preprocess_data(text): 
 # lowercase, tokenize, and remove stopwords
    doc = nlp(text)
    verbs = [token.text.lower() for token in doc if token.pos_ == "VERB"]
    preprocessed_text = simple_preprocess(text)
    return [word for word in preprocessed_text if word not in verbs and word not in stop_words]

In [48]:
preprocessed_data = data.apply(preprocess_data).to_list()

In [49]:
# Create a Dictionary: a mapping between words and their integer IDs
id2word = corpora.Dictionary(preprocessed_data)

# Create a corpus: a list of documents represented as a BoW
corpus = [id2word.doc2bow(text) for text in preprocessed_data]

In [61]:
# Build the LDA model 
num_topics = 3
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha='auto', per_word_topics=True)

In [63]:
# Print the keywords for each topic
# lda_model.print_topics()

In [64]:
# Evaluation
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=preprocessed_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.6074114916865837


In [65]:
# Visualization 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()

In [66]:
vis = gensimvis.prepare(lda_model, corpus, id2word) 

In [67]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.082935  0.067544       1        1  36.850632
0      0.026917 -0.095210       2        1  33.101064
2     -0.109852  0.027665       3        1  30.048304, topic_info=                Term       Freq      Total Category  logprob  loglift
203     elektrischen   7.000000   7.000000  Default  30.0000  30.0000
187               km   6.000000   6.000000  Default  29.0000  29.0000
2               satz  11.000000  11.000000  Default  28.0000  28.0000
136            nenne   8.000000   8.000000  Default  27.0000  27.0000
197  geschwindigkeit   6.000000   6.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
97             farbe   1.370967   2.390666   Topic3  -5.2999   0.6463
316           südpol   1.370802   2.390670   Topic3  -5.3001   0.6462
314          magnete   1.370793   3.654775   Topic3  -5.3001   0.2217
242             erde   1.370596   2.390679   Topic3  -5.3002   0.6460
204      stromstärke   1.370515   3.048094   Topic3  -5.3003   0.4030

[166 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
60        1  0.805941      ach
146       1  0.407470  aussage
146       2  0.407470  aussage
183       1  0.204871     auto
183       2  0.819485     auto
...     ...       ...      ...
199       2  0.823448      zug
191       2  0.548253   zurück
19        1  0.272199     zwei
19        2  0.272199     zwei
19        3  0.272199     zwei

[161 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])